In [1]:
import pandas as pd
import numpy as np

## Features engineering de conversion

### Carga de datos

In [2]:
data = pd.read_pickle("../data/conversiones.pkl")

---

### Cargo features generados hasta el momento


In [3]:
# Trimestre 2 - 3 - 4 para los que tengo datos anteriores
CANT_TRIMESTRES = 4

In [4]:
TRIMESTRES = {}
trim_no_encontrados =[]

In [5]:
def generar_dataframes_ventana(numero_ventana,dataframe):
    
    if numero_ventana == 4 :   #Para la ventana 4 se necesitan los 11 mil usuarios existentes
        df = pd.read_pickle("../data/users.pkl")
        df.set_index("USER_ID",inplace = True)
    
    else:    
        condicion = dataframe["trimestre"] == numero_ventana
        df = dataframe[condicion]["USER_ID"].drop_duplicates().to_frame().set_index("USER_ID").copy()

    return df.sort_index()
    

In [6]:
print("Cargando features")
for ventana_nro in range(1,CANT_TRIMESTRES) : #Trimestres del 1 al 3
    try:
        TRIMESTRES[ventana_nro] = pd.read_pickle("../data/features/ventana_{}_conversion.pkl".format(ventana_nro))
        
    except FileNotFoundError:
        print("No se encontraron los features, se generarán los dataframe requeridos")
        trim_no_encontrados.append(ventana_nro)
    continue
    
for ventana_nro in trim_no_encontrados:    
    TRIMESTRES[ventana_nro] = generar_dataframes_ventana(ventana_nro,data)

Cargando features


### Set para predecir

In [7]:
try:
    print("Cargando features para predecir")
    TRIMESTRES[4] = pd.read_pickle("../data/features/predecir_conversion.pkl")
except FileNotFoundError:
    print("No se encontro el feature deseado, se generara el dataframe requerido")
    TRIMESTRES[4] = generar_dataframes_ventana(4,data)

Cargando features para predecir


---

### Diccionario de features

In [8]:
#Posicion dentro del diccionario de features que ocupa la lista de columnas generadas por el mismo
POS_COLUMNAS_DEL_FEATURE = 0
POS_TIPO_DATO = 1
POS_DATO_NAN = 2

In [9]:
# Clave = nombre_feature : Valor = funcion
FUNCIONES_FEATURES = {}
# Clave = nombre_feature : Valor = (columnas,tipo_dato,valor_nan)
DATOS_FEATURES = {}

In [10]:
FEATURES_YA_GENERADOS = {}

for ventana_nro in range (1, CANT_TRIMESTRES + 1):
    
    FEATURES_YA_GENERADOS[ventana_nro] = list(TRIMESTRES[ventana_nro].columns.values)

---


---

## Creación de features

#### Vistazo del dataset: __Conversiones__

In [11]:
data.head()

,mes,anio,USER_ID,trimestre
0,1,2018,2483,1
1,1,2018,7088,1
2,1,2018,1787,1
3,1,2018,2085,1
4,1,2018,8270,1


---

### Conviritio en trimestre

In [12]:
nombre = "convirtio en trimestre"

In [13]:
def conversiones_en_trimestre(dataframe):
    """ Para cada columna generada:
            0-> no convirtio ese trimestre
            1-> convirtio ese trimestre
    """
    
    conversiones_trimestre = dataframe.groupby(["USER_ID","trimestre"]).agg({"anio":"nunique"})  
    
    conversiones_trimestre.reset_index(inplace = True)
    
    conversiones_trimestre = conversiones_trimestre.pivot(index = "USER_ID", columns = "trimestre", values = "anio")
    conversiones_trimestre.columns = ["convirtio_en_trimestre_actual"]
    


    return conversiones_trimestre

In [14]:
FUNCIONES_FEATURES[nombre] = conversiones_en_trimestre

In [15]:
columnas_generadas = ["convirtio_en_trimestre_actual"]
tipo_dato = "int8"
dato_nan = 0

DATOS_FEATURES[nombre] = (columnas_generadas,tipo_dato,dato_nan) 

#### Conversiones a traves de los meses

In [16]:
nombre = "conversiones en los meses"

In [17]:
def conversiones_en_meses(dataframe):
    """ Para cada columna generada:
            0-> no convirtio ese mes
            1-> convirtio ese mes
    """
    
    conversiones_meses = dataframe.groupby(["USER_ID","mes"]).agg({"trimestre":"nunique"})
    conversiones_meses.reset_index(inplace = True)
    conversiones_meses = conversiones_meses.pivot(index = "USER_ID", columns = "mes", values = "trimestre")
    
     #lleno los meses vacios con 0
    conversiones_meses.columns = [x for x in range(1,4)]
    for mes in range(1,4):
        try:             #si no existe el mes, lo creo
            conversiones_meses[mes].fillna(0) 
        except KeyError:
            conversiones_meses.insert(mes,mes,0)

    
    conversiones_meses.columns = ["convirtio_en_mes_{}".format(mes) for mes in range(1,4)]

    return conversiones_meses

In [18]:
FUNCIONES_FEATURES[nombre] = conversiones_en_meses

In [19]:
columnas_generadas = ["convirtio_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int8"
dato_nan = 0

DATOS_FEATURES[nombre] = (columnas_generadas,tipo_dato,dato_nan) 

---

#### ¿Convirtio mas de una vez por mes?

In [20]:
nombre = "conversiones multiples por mes"

In [21]:
def conversiones_multiples_en_meses(dataframe):
    """ Para cada columna generada:
            0-> no convirtio multiples veces
            1-> convirtio multiples veces
    """
    
    conversiones_meses = dataframe.groupby(["USER_ID","mes"]).agg({"trimestre":"count"})
    conversiones_meses.reset_index(inplace = True)
    conversiones_meses = conversiones_meses.pivot(index = "USER_ID", columns = "mes", values = "trimestre")
  
    #lleno los meses vacios con 0
    conversiones_meses.columns = [x for x in range(1,4)]
    for mes in range(1,4):
        try:             #si no existe el mes, lo creo
            conversiones_meses[mes].fillna(0) 
        except KeyError:
            conversiones_meses.insert(mes,mes,0)

    
    conversiones_meses.columns = ["convirtio_multiple_en_mes_{}".format(mes) for mes in range(1,4)]

    return conversiones_meses

In [22]:
FUNCIONES_FEATURES[nombre] = conversiones_multiples_en_meses

In [23]:
columnas_generadas = ["convirtio_multiple_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int8"
dato_nan = 0

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

---

## Generacion de features

In [24]:
def feature_ya_generado(nombre_feature, trimestre):
    """Corrobora si un feature ya fue generado """
    
    #Obtengo las columnas que genera ese feature
    columnas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
    for columna in columnas:
        if columna in FEATURES_YA_GENERADOS[trimestre]:
            return True
    return False

In [25]:
def generar_feature_en_ventana(dataframe,trimestre, nombre_feature, destino):
    """Genera un feature en la ventana correspondiente, con los tipos de datos correspondientes"""
    
    #Calculo el feature
    funcion_feature = FUNCIONES_FEATURES[nombre_feature]
    dataframe = funcion_feature( dataframe[dataframe["trimestre"] == trimestre] )
    
    #Uno los features nuevos con los viejos
    dataframe = destino[trimestre].merge(dataframe, left_index = True, right_index= True, how = 'left')
    
    #Rello los nans, si es que los hubiera
    dataframe = dar_formato_al_feature_nuevo(dataframe,nombre_feature)
    
    destino[trimestre] = dataframe

In [26]:
def dar_formato_al_feature_nuevo(dataframe, nombre_feature):
    """Rellena los Nans de un dataframe con features nuevos"""
    
    
    #Datos para rellenar los Nan
    columnas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
    tipo_dato= DATOS_FEATURES[nombre_feature][POS_TIPO_DATO]
    valor_nan = DATOS_FEATURES[nombre_feature][POS_DATO_NAN]
    
    dataframe.fillna(valor_nan, inplace = True)
    dataframe[columnas] = dataframe[columnas].astype(tipo_dato)
    
    return dataframe
    

In [27]:
def generar_features_faltantes(dataframe,destino,trimestre):
    """
    Funcion para generar los features faltantes para un trimestre, 
    evita la repeticion de calculo de features si estos ya se encuentran generados.
    
    dataframe: Dataframe a partir de cual se calcularan los features.
    destino: Diccionario de ventanas para los que calcular los features.
    trimestre: Numero de ventana a partir de la cual se calculan los features.
    """
    
    for nombre_feature in FUNCIONES_FEATURES:    
        if feature_ya_generado(nombre_feature, trimestre):
            continue

        #Si no fueron generados, los genero y actualizo el listado:   
        generar_feature_en_ventana(dataframe,trimestre, nombre_feature, destino)   
        
        #Agrego las columnas nuevas al diccionario
        columnas_generadas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
        FEATURES_YA_GENERADOS[trimestre].extend(columnas_generadas)

In [28]:
for ventana_nro in range(1, CANT_TRIMESTRES + 1):
    generar_features_faltantes(data, TRIMESTRES, ventana_nro)

In [29]:
TRIMESTRES[1].head()

,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3,convirtio_en_trimestre_actual
USER_ID,,,,,,,
40,1,0,0,1,0,0,1
92,0,0,1,0,0,1,1
204,0,1,0,0,1,0,1
351,1,0,0,1,0,0,1
381,0,1,0,0,1,0,1


#### Es normal que el head tenga 0, .sum() para corroborar que no son todos 0

In [30]:
TRIMESTRES[4].head()

,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3,convirtio_en_trimestre_actual
USER_ID,,,,,,,
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


In [31]:
TRIMESTRES[4].sum()

convirtio_en_mes_1               186
convirtio_en_mes_2               166
convirtio_en_mes_3               101
convirtio_multiple_en_mes_1      210
convirtio_multiple_en_mes_2      184
convirtio_multiple_en_mes_3      115
convirtio_en_trimestre_actual    391
dtype: int64

##### No son todos 0

### Guardado

In [32]:
for ventana_nro in range(1, CANT_TRIMESTRES ):
    TRIMESTRES[ventana_nro].to_pickle("../data/features/ventana_{}_conversion.pkl".format(ventana_nro))

TRIMESTRES[4].to_pickle("../data/features/predecir_conversion.pkl")